In [344]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Hourly
import numpy as np
import pandas as pd
import geopy.distance
import glob

In [8]:
data = pd.read_csv("../processed/processed-shutoffs.csv")
data.head()

,circuit_name,deenergize_time,restoration_time,time_out_min,key_communities,hftd_tier,total_affected,residential_affected,zip_code,longitude,latitude,substn_present,total_population
0,APPLE HILL-1103,2018-10-14 21:00:00,2018-10-15 16:29:00.000000000,1169.0,"Camino, Placerville",3.0,1809.0,1488.0,95709,-120.677165,38.739683,True,5634.0
1,APPLE HILL-1104,2018-10-14 21:17:00,2018-10-15 18:37:00.000000000,1280.0,"Camino, Placerville, Pollock Pines,",3.0,2261.0,2043.0,95709,-120.677165,38.739683,True,5634.0
2,APPLE HILL-2102,2018-10-14 21:05:00,2018-10-16 18:04:00.000000000,2699.0,"Camino, Grizzly Flats, Mount Aukum, Placervill...",3.0,4489.0,4013.0,95709,-120.677165,38.739683,True,5634.0
3,BONNIE NOOK-1102,2018-10-14 21:54:00,2018-10-15 18:04:00.000000000,1210.0,Alta,3.0,533.0,454.0,95701,-120.809292,39.214708,True,1064.0
4,CALISTOGA-1101,2018-10-14 20:34:00,2018-10-15 21:27:00.000000000,1493.0,Calistoga,3.0,1596.0,1216.0,94515,-122.578610,38.592088,True,7525.0


In [21]:
data["deenergize_time"] = pd.to_datetime(data["deenergize_time"])
data["restoration_time"] = pd.to_datetime(data["restoration_time"])
data["deenergize_date"] = data["deenergize_time"].dt.date
data["restoration_date"] = data["restoration_time"].dt.date


In [370]:
missing = {}
full_data = {}
weather_present = {}
tmin_missing = {}
tmax_missing = {}
wspd_missing = {}
daily = {}
for i in range(data.shape[0]):
    point_1 = Point(data.loc[i, "latitude"], data.loc[i, "longitude"])
    start = pd.to_datetime(data.loc[i, "deenergize_date"]) - timedelta(days=5)
    end = pd.to_datetime(data.loc[i, "deenergize_date"])
    # weather = Hourly(point_1, start, end)
    weather = Daily(point_1, start, end)
    weather = weather.fetch()
    # if weather.shape[0] == 0:
    #     weather = Daily(point_1, start, end)
    #     weather = weather.fetch()
    if weather.shape[0] == 0:
        missing[i] = data.loc[i, ["circuit_name", "deenergize_time", "latitude", "longitude"]]
    #     daily[i] = weather
    elif weather.shape[0] >= 5:
        if weather.shape[0] > 5:
            weather = weather.loc[start.date(): end.date() - timedelta(days=1), :]
            assert weather.shape[0] == 5
        if sum(weather["tmin"].isna()) > 0:
            tmin_missing[i] = data.loc[i, ["circuit_name",
                                "deenergize_time", "latitude", "longitude"]]
        if sum(weather["tmax"].isna()) > 0:
            tmax_missing[i] = data.loc[i, ["circuit_name",
                               "deenergize_time", "latitude", "longitude"]]
        if sum(weather["wspd"].isna()) > 0:
            wspd_missing[i] = data.loc[i, ["circuit_name", "deenergize_time", "latitude", "longitude"]]
        if sum(np.sum(weather[["tmin", "tmax", "wspd"]].isna(), axis=1)) == 0:
            full_data[i] = data.loc[i, ["circuit_name", "deenergize_time", "latitude", "longitude"]]
            weather["circuit_name"] = data.loc[i, "circuit_name"]
            weather["deenergize_time"] = data.loc[i, "deenergize_time"]
            weather["source_circuit_name"] = data.loc[i, "circuit_name"]
            weather["source_deenergize_time"] = data.loc[i, "deenergize_time"]
            weather["approximated"] = False
            daily[i] = weather
    else:
        weather_present[i] = data.loc[i, ["circuit_name", "deenergize_time", "latitude", "longitude"]]
        

In [371]:
full_data = pd.DataFrame(full_data).T
full_data.to_csv("full_weather.csv")
missing = pd.DataFrame(missing).T
missing.to_csv("missing_weather.csv")
weather_present = pd.DataFrame(weather_present).T
weather_present.to_csv("some_missing_weather.csv")
tmin_missing = pd.DataFrame(tmin_missing).T
tmin_missing.to_csv("tmin_missing.csv")
tmax_missing = pd.DataFrame(tmax_missing).T
tmax_missing.to_csv("tmax_missing.csv")
wspd_missing = pd.DataFrame(wspd_missing).T
wspd_missing.to_csv("wspd_missing.csv")


In [427]:
for i in list(daily.keys()):
    shutoff = data.iloc[i, :]
    filename = f"raw/daily/weather_{i}_{shutoff['circuit_name']}_{shutoff['deenergize_date']}.csv"
    weather = daily[i]
    assert os.path.isfile(filename) == False
    weather.to_csv(filename)
    

In [373]:
# for i in list(hourly.keys()):
#     shutoff = data.iloc[i, :]
#     filename = f"raw/hourly/weather_{shutoff['circuit_name']}_{shutoff['deenergize_date']}_hourly.csv"
#     weather = hourly[i]
#     weather.to_csv(filename)


In [483]:
# Replace stations that are entirely missing weather data with a copy
# of their nearest neighbor's weather

knn_missing = {}
to_drop = {}
for i in missing.index:
    de_time = missing.loc[i, "deenergize_time"].date()
    dist = full_data[full_data["deenergize_time"].dt.date == de_time].apply(lambda row: geopy.distance.geodesic(
        (row["latitude"], row["longitude"]),
        (missing.loc[i, "latitude"],
         missing.loc[i, "longitude"])).miles,
        axis=1)
    if len(dist) == 0:
        to_drop[i] = missing.loc[i]
        continue
    min_idx = dist.idxmin()
    dist = min(dist)
    if dist > 60:
        to_drop[i] = missing.loc[i]
        continue
    knn_missing[i] = {
        "distance": dist,
        "min_idx": min_idx,
        "closest_circuit_name": full_data.loc[min_idx].loc["circuit_name"],
        "closest_deenergize_time": full_data.loc[min_idx].loc["deenergize_time"],
        "closest_latitude": full_data.loc[min_idx].loc["latitude"],
        "closest_longitude": full_data.loc[min_idx].loc["longitude"]
    }


In [506]:
# Join to get rest for information
knn_missing = pd.DataFrame(knn_missing).T.join(missing)

# Create copy of weather from nearest neighbor, write to disk
for i in knn_missing.index:
    file_name = glob.glob(
        f"raw/daily/weather_{knn_missing['min_idx'].loc[i]}_{knn_missing['closest_circuit_name'].loc[i]}_{str(knn_missing['closest_deenergize_time'].loc[i].date())}.csv")
    assert len(file_name) == 1
    curr_weather = pd.read_csv(file_name[0])
    curr_weather["circuit_name"] = knn_missing['circuit_name'].loc[i]
    curr_weather["deenergize_time"] = knn_missing['deenergize_time'].loc[i]
    curr_weather["source_circuit_name"] = knn_missing['closest_circuit_name'].loc[i]
    curr_weather["source_deenergize_time"] = knn_missing['closest_deenergize_time'].loc[i]
    curr_weather["approximated"] = True
    curr_weather.to_csv(
        f"raw/daily/weather_{i}_{knn_missing['circuit_name'].loc[i]}_{str(knn_missing['deenergize_time'].loc[i].date())}.csv")


In [509]:
len(to_drop)

36

In [508]:
len(os.listdir("raw/daily"))

1556

In [510]:
data.shape[0]

2137

In [511]:
1556/2137

0.7281235376696303

In [522]:
remaining_idx = list(tmax_missing.index).append(
    list(tmin_missing.index))


In [ ]:
remaining_idx.append(list(wspd_missing.index))


In [517]:
set.intersection(*map(set,d))

{190,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 221,
 223,
 224,
 248,
 265,
 266,
 271,
 272,
 289,
 290,
 291,
 367,
 368,
 369,
 456,
 584,
 585,
 588,
 589,
 590,
 591,
 594,
 595,
 609,
 622,
 626,
 627,
 628,
 629,
 653,
 703,
 719,
 727,
 728,
 742,
 743,
 751,
 752,
 753,
 754,
 755,
 771,
 772,
 802,
 803,
 804,
 805,
 806,
 828,
 829,
 833,
 834,
 835,
 836,
 837,
 838,
 839,
 840,
 841,
 842,
 851,
 852,
 853,
 884,
 885,
 886,
 887,
 888,
 889,
 894,
 895,
 896,
 897,
 907,
 908,
 909,
 936,
 939,
 940,
 974,
 975,
 984,
 985,
 986,
 987,
 988,
 1016,
 1065,
 1066,
 1070,
 1076,
 1077,
 1163,
 1164,
 1349,
 1441,
 1442,
 1491,
 1492,
 1506,
 1507,
 1508,
 1509,
 1510,
 1511,
 1512,
 1533,
 1534,
 1563,
 1594,
 1595,
 1596,
 1597,
 1600,
 1601,
 1602,
 1603,
 1604,
 1740,
 1741,
 1787,
 1788,
 2044,
 2101,
 2102}

In [173]:
# Replace stations that are missing some rows of weather with:
# (a) copy of missing rows from nearest neighbor
# (b) copy of missing weather data with a copy
# of their nearest neighbor's weather

knn_some_missing = {}
for i in range(weather_present.shape[0]):
    dist = full_data.apply(lambda row: geopy.distance.geodesic(
        (row["latitude"], row["longitude"]),
        (weather_present.iloc[i, :].loc["latitude"],
         weather_present.iloc[i, :].loc["longitude"])),
        axis=1)
    min_idx = np.argmin(dist)
    dist = min(dist)
    knn_some_missing[i] = {
        "distance": dist,
        "min_idx": min_idx
    }
